# Fox Lab NWB Reader

This notebook demonstrates how to read and explore NWB files created by the Fox Lab's conversion script.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pynwb import read_nwb
from pathlib import Path

## Load the NWB file

First, specify the path to your NWB file and load it using the read_nwb function.

In [ ]:
# Replace with the path to your NWB file
nwbfile_path = Path("/home/heberto/conversion_nwb/Fly3_240124_115923.nwb")

# Open the NWB file
nwbfile = read_nwb(nwbfile_path)
nwbfile

## Explore file metadata

Let's first look at the general metadata in the NWB file.

In [ ]:
# Display general file metadata
print(f"NWB File: {nwbfile.identifier}")
print(f"Session ID: {nwbfile.session_id}")
print(f"Session Description: {nwbfile.session_description}")
print(f"Session Start Time: {nwbfile.session_start_time}")

# Display subject information if available
if nwbfile.subject is not None:
    print("\nSubject Information:")
    print(f"Subject ID: {nwbfile.subject.subject_id}")
    if hasattr(nwbfile.subject, 'age'):
        print(f"Age: {nwbfile.subject.age}")
    if hasattr(nwbfile.subject, 'sex'):
        print(f"Sex: {nwbfile.subject.sex}")
    if hasattr(nwbfile.subject, 'genotype'):
        print(f"Genotype: {nwbfile.subject.genotype}")

## Explore devices and camera metadata

The Fox Lab NWB file contains metadata about the cameras and other devices used for recording.

In [ ]:
# Display devices
print("Devices:")
for device_name, device in nwbfile.devices.items():
    print(f"- {device_name}")
    if hasattr(device, 'description') and device.description is not None:
        print(f"  Description: {device.description}")
    if hasattr(device, 'manufacturer') and device.manufacturer is not None:
        print(f"  Manufacturer: {device.manufacturer}")

## Explore video data

The Fox Lab NWB file contains multiple synchronized camera recordings (side, top, and haltere views).

In [ ]:
# Check for image series data in acquisition
video_series_names = [name for name in nwbfile.acquisition if 'video' in name.lower() or 'cam' in name.lower()]

if video_series_names:
    print("Available video data in acquisition:")
    for name in video_series_names:
        data = nwbfile.acquisition[name]
        print(f"- {name}")
        if hasattr(data, 'description') and data.description is not None:
            print(f"  Description: {data.description}")
        
        # Check if it's an external file
        if hasattr(data, 'external_file') and data.external_file is not None:
            print(f"  External file: {data.external_file}")
            print(f"  Format: {data.format}")
        elif hasattr(data, 'data') and data.data is not None:
            print(f"  Data shape: {data.data.shape}")
        
        # Check for timestamps
        if hasattr(data, 'timestamps') and data.timestamps is not None:
            print(f"  Number of timestamps: {len(data.timestamps)}")
            print(f"  First timestamp: {data.timestamps[0]}")
            print(f"  Last timestamp: {data.timestamps[-1]}")
        elif hasattr(data, 'rate') and data.rate is not None:
            print(f"  Rate: {data.rate} Hz")
else:
    print("No video data found in acquisition.")

## Explore wing kinematics data

The Fox Lab NWB file contains detailed wing kinematics data including wing beat amplitude, frequency, and voltage recordings.

In [ ]:
# List all acquisition data related to wing kinematics
wing_data_names = [name for name in nwbfile.acquisition if 'wing' in name.lower()]

if wing_data_names:
    print("Available wing kinematics data:")
    for name in wing_data_names:
        data = nwbfile.acquisition[name]
        print(f"- {name}")
        if hasattr(data, 'description') and data.description is not None:
            print(f"  Description: {data.description}")
        if hasattr(data, 'unit') and data.unit is not None:
            print(f"  Unit: {data.unit}")
        if hasattr(data, 'data') and data.data is not None:
            print(f"  Data shape: {data.data.shape}")
        if hasattr(data, 'conversion') and data.conversion is not None:
            print(f"  Conversion factor: {data.conversion}")
else:
    print("No wing kinematics data found in the file.")

In [ ]:
# Plot wing kinematics data
if wing_data_names:
    plt.figure(figsize=(12, 8))
    for i, name in enumerate(wing_data_names):
        data = nwbfile.acquisition[name]
        plt.subplot(len(wing_data_names), 1, i+1)
        
        # Get timestamps and data
        timestamps = data.timestamps[:] if data.timestamps is not None else np.arange(len(data.data))
        
        # Plot a subset of the data to avoid overwhelming the plot
        max_points = 10000
        if len(timestamps) > max_points:
            step = len(timestamps) // max_points
            plt.plot(timestamps[::step], data.data[::step])
        else:
            plt.plot(timestamps, data.data[:])
        
        plt.title(name)
        plt.xlabel('Time (s)')
        plt.ylabel(f'{data.unit}')
    
    plt.tight_layout()
    plt.show()
else:
    print("No wing kinematics data found to plot.")

## Explore DeepLabCut pose estimation data

The Fox Lab NWB file contains DeepLabCut pose estimation data for tracking body parts.

In [ ]:
# Check if the behavior processing module exists
if "behavior" in nwbfile.processing:
    # Get the behavior processing module
    processing_modules = nwbfile.processing["behavior"]
    
    # List all data interfaces in the behavior module
    print("Data interfaces in behavior module:")
    for name, interface in processing_modules.data_interfaces.items():
        print(f"- {name} ({type(interface).__name__})")
    
    # Find pose estimation modules
    pose_estimation_modules = [module for name, module in processing_modules.data_interfaces.items() if "Pose" in name]
    
    if pose_estimation_modules:
        # Get the first pose estimation module
        pose_interface = pose_estimation_modules[0]
        
        # Get the pose estimation series
        pose_estimation_series = pose_interface.pose_estimation_series
        
        # List all pose estimation series
        print("\nAvailable pose estimation series:")
        for name in pose_estimation_series.keys():
            print(f"- {name}")
    else:
        print("No pose estimation modules found in the behavior module.")
else:
    print("No behavior processing module found in the file.")

In [ ]:
# Plot pose estimation data for a specific body part
if "behavior" in nwbfile.processing:
    # Get the behavior processing module
    processing_modules = nwbfile.processing["behavior"]
    
    # Find pose estimation modules
    pose_estimation_modules = [module for name, module in processing_modules.data_interfaces.items() if "Pose" in name]
    
    if pose_estimation_modules:
        # Get the first pose estimation module
        pose_interface = pose_estimation_modules[1]
        
        # Get the pose estimation series
        pose_estimation_series = pose_interface.pose_estimation_series
        
        if pose_estimation_series:
            # Get the first series name
            series_name = list(pose_estimation_series.keys())[0]
            series = pose_estimation_series[series_name]
            
            # Get timestamps and data
            timestamps = series.timestamps[:] if series.timestamps is not None else np.arange(len(series.data))
            data = series.data[:]
            
            # Plot X and Y coordinates
            plt.figure(figsize=(12, 6))
            
            plt.subplot(2, 1, 1)
            plt.plot(timestamps, data[:, 0])
            plt.title(f"{series_name} - X Coordinate")
            plt.xlabel('Time (s)')
            plt.ylabel('Position')
            
            plt.subplot(2, 1, 2)
            plt.plot(timestamps, data[:, 1])
            plt.title(f"{series_name} - Y Coordinate")
            plt.xlabel('Time (s)')
            plt.ylabel('Position')
            
            plt.tight_layout()
            plt.show()
            
            # Plot trajectory (X vs Y)
            plt.figure(figsize=(8, 8))
            plt.plot(data[:, 0], data[:, 1])
            plt.title(f"{series_name} - Trajectory")
            plt.xlabel('X Position')
            plt.ylabel('Y Position')
            plt.axis('equal')
            plt.grid(True)
            plt.show()
        else:
            print("No pose estimation series found.")
    else:
        print("No pose estimation modules found in the behavior module.")
else:
    print("No behavior processing module found in the file.")